In [1]:
import pandas as pd
import joblib
from etl_model_utils import predict_column_transformation

In [2]:
df = pd.read_csv('K:/ETL_Automation/ETLAutomation/data/test/Social_Media_Engagement_Dataset.csv')

In [3]:
df.dtypes

post_id                     object
timestamp                   object
day_of_week                 object
platform                    object
user_id                     object
location                    object
language                    object
text_content                object
hashtags                    object
mentions                    object
keywords                    object
topic_category              object
sentiment_score            float64
sentiment_label             object
emotion_type                object
toxicity_score             float64
likes_count                  int64
shares_count                 int64
comments_count               int64
impressions                  int64
engagement_rate            float64
brand_name                  object
product_name                object
campaign_name               object
campaign_phase              object
user_past_sentiment_avg    float64
user_engagement_growth     float64
buzz_change_rate           float64
dtype: object

In [4]:


model = joblib.load("../models/column_transform_model.pkl")
label_encoder = joblib.load("../models/label_encoder.pkl")

def smart_etl_with_model(filepath, model):
    df = pd.read_csv(filepath)
    transformed_df = df.copy()

    for col in df.columns:
        action = predict_column_transformation(df[col], model, label_encoder)
        print(f"{col}: {action}")

        if action == 'drop_column':
            transformed_df.drop(columns=[col], inplace=True)
        elif action == 'fill_zero':
            transformed_df[col] = transformed_df[col].fillna(0)
        elif action == 'fill_mean':
            if pd.api.types.is_numeric_dtype(df[col]):
                transformed_df[col] = transformed_df[col].fillna(df[col].mean())
        elif action == 'strip_string':
            transformed_df[col] = transformed_df[col].astype(str).str.strip()
        elif action == 'cast_to_string':
            transformed_df[col] = transformed_df[col].astype(str)
        elif action == 'fill_mode':
            mode_val = df[col].mode(dropna=True)
            if not mode_val.empty:
                transformed_df[col] = df[col].fillna(mode_val[0])
        elif action == 'fill_mode_date':
            if pd.api.types.is_datetime64_any_dtype(df[col]):
                mode_val = df[col].mode(dropna=True)
                if not mode_val.empty:
                    transformed_df[col] = df[col].fillna(mode_val[0])
        elif action == 'cast_to_int':
            transformed_df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert to float, NaNs where conversion fails
            transformed_df[col] = transformed_df[col].fillna(0).astype(int)  # Fill NaNs (e.g., with 0) and cast to int

    #print("\n✅ Final transformed data:")
    #print(transformed_df.head())
    return transformed_df

# Example usage:
df = smart_etl_with_model("../data/test/Social_Media_Engagement_Dataset.csv", model)


post_id: drop_column
timestamp: drop_column
day_of_week: nan
platform: nan
user_id: drop_column
location: drop_column
language: nan
text_content: drop_column
hashtags: drop_column
mentions: fill_mode_date
keywords: drop_column
topic_category: nan
sentiment_score: nan
sentiment_label: nan
emotion_type: nan
toxicity_score: nan
likes_count: nan
shares_count: nan
comments_count: nan
impressions: nan
engagement_rate: nan
brand_name: nan
product_name: nan
campaign_name: drop_column
campaign_phase: nan
user_past_sentiment_avg: nan
user_engagement_growth: nan
buzz_change_rate: nan


In [5]:
df.head()

,day_of_week,platform,language,mentions,topic_category,sentiment_score,sentiment_label,emotion_type,toxicity_score,likes_count,shares_count,comments_count,impressions,engagement_rate,brand_name,product_name,campaign_phase,user_past_sentiment_avg,user_engagement_growth,buzz_change_rate
0,Monday,Instagram,pt,NaN,Pricing,0.9826,Positive,Confused,0.0376,1264,1704,701,18991,0.19319,Google,Chromebook,Launch,0.0953,-0.3672,19.1
1,Sunday,Twitter,ru,"@CustomerService, @BrandCEO",Delivery,-0.3793,Negative,Angry,0.9715,522,1803,359,52764,0.05086,Microsoft,Surface Laptop,Post-Launch,0.1369,-0.4510,-42.6
2,Saturday,Reddit,ru,NaN,Product,-0.3596,Negative,Confused,0.5124,2689,705,643,8887,0.45425,Nike,Epic React,Post-Launch,0.2855,-0.4112,17.4
3,Monday,YouTube,en,"@StyleGuide, @BrandSupport",Delivery,-0.7282,Negative,Sad,0.4002,1827,262,743,6696,0.42293,Pepsi,Diet Pepsi,Launch,-0.2094,-0.0167,-5.5
4,Thursday,Twitter,hi,"@BrandSupport, @InfluencerName",Product,0.5460,Positive,Happy,0.0862,2005,1443,703,47315,0.08773,Toyota,Corolla,Launch,0.6867,0.0807,38.8


In [6]:
df.dtypes

day_of_week                 object
platform                    object
language                    object
mentions                    object
topic_category              object
sentiment_score            float64
sentiment_label             object
emotion_type                object
toxicity_score             float64
likes_count                  int64
shares_count                 int64
comments_count               int64
impressions                  int64
engagement_rate            float64
brand_name                  object
product_name                object
campaign_phase              object
user_past_sentiment_avg    float64
user_engagement_growth     float64
buzz_change_rate           float64
dtype: object